## Loading data

In [1]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip

import os
import zipfile

local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

--2024-11-23 03:15:53--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.0.91, 172.217.7.59, 172.217.12.27, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.0.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M   129MB/s    in 0.5s    

2024-11-23 03:15:54 (129 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



## Reading the Data into arrays

In [15]:
from PIL import Image
import numpy as np
import os

cats_dir = os.path.join(train_dir, "cats")
dogs_dir = os.path.join(train_dir, "dogs")
x_train = []
y_train = []
NUMBER_OF_EXAMPLES = min(len(os.listdir(cats_dir)), len(os.listdir(dogs_dir))) * 2

i = 0
while i < NUMBER_OF_EXAMPLES:
    if i % 2 == 0:

        im = Image.open(os.path.join(cats_dir, os.listdir(cats_dir)[i // 2])).convert("RGB")
        im_resized = im.resize((150, 150))
        x_train.append(np.array(im_resized))
        y_train.append(1)
    else:

        im = Image.open(os.path.join(dogs_dir, os.listdir(dogs_dir)[i // 2])).convert("RGB")
        im_resized = im.resize((150, 150))
        x_train.append(np.array(im_resized))
        y_train.append(0)
    i += 1

## Beginning to define the model (this is where you come in, I loaded the pretrained model for you)

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import preprocess_input


pretrained_model = tf.keras.applications.ResNet50(
    include_top=False,
    input_shape=(150, 150, 3),
    pooling='avg',
    classes=2,
    weights='imagenet'
)
x_train = np.array(x_train, dtype="float32")
y_train = np.array(y_train, dtype="float32")

x_train = preprocess_input(x_train)


split = int(len(x_train) * 0.8)
x_train, x_val = x_train[:split], x_train[split:]
y_train, y_val = y_train[:split], y_train[split:]
model = tf.keras.Sequential([
    pretrained_model,
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)


history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=10,
    batch_size=32
)


model.evaluate(x_val, y_val)
model.save("cats_vs_dogs_classifier.h5")

Epoch 1/10
 6/50 ━━━━━━━━━━━━━━━━━━━━ 8:08 11s/step - accuracy: 0.6061 - loss: 0.6844